In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from scripts.gestion_donnees.pretraitement import Pretraitement
from scripts.evaluation.evaluation_model import Evaluation
#import sys
#sys.path.insert(1,'C:\\Users\\mazen\\OneDrive\\Bureau\\Projet_IFT712\\src\\scripts\\evaluation')
#import evaluation_model
#evaluation_model.Evaluation.matrice_confusion()


## Partie 1 :  Prétraitement et analyse des données

In [2]:
# creation des instances
Ev = Evaluation
Pt = Pretraitement("./data/train.csv")

In [3]:
# charger donneés entrainement
df_train_init = Pt.Charger_donnees()
# encoder donneés
etiquette,classes = Pt.Encoder_donnees(df_train_init,"species")
df_train_init = df_train_init.drop(['id','species'], axis=1)
# supprimer les valeurs aberrantes
outliers = Pt.Indice_outliers(df_train_init,3.0)
df_train = df_train_init.drop(outliers)
etiquette = [etiquette[i] for i in range(len(etiquette)) if i not in outliers]
# normalisation des données
print('zebi')
df_train
df_train = Pt.Normalize_dataframe(df_train)
# reduction de dimensions
print('df_train000000000000000000000')
df_train
# diviser les donnees
#x_train, x_test, y_train, y_test, x_val, y_val = Pt.Diviser_donnees(df_train,etiquette)


df_train000000000000000000000


,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.088883,0.114287,0.150003,0.022987,0.105264,0.031447,0.325586,0.0,0.025639,0.340000,...,0.018181,0.000000,0.016951,0.014635,0.330258,0.000000,0.000000,0.012987,0.000000,0.179315
1,0.066662,0.000000,0.200000,0.091955,0.228070,0.006289,0.232556,0.0,0.000000,0.079995,...,0.002274,0.000000,0.000000,0.004880,0.220178,0.000000,0.000000,0.002599,0.449433,0.158623
2,0.066662,0.047620,0.124998,0.045975,0.035085,0.018867,0.813953,0.0,0.000000,0.460002,...,0.359096,0.000000,0.033896,0.004880,0.073387,0.000000,0.000000,0.000000,0.235957,0.020692
3,0.000000,0.019046,0.150003,0.034481,0.192976,0.062892,0.279077,0.0,0.179489,0.179999,...,0.000000,0.004833,0.000000,0.000000,0.192654,0.000000,0.000000,0.046752,0.000000,0.337938
4,0.066662,0.019046,0.312499,0.057474,0.122806,0.050314,0.069763,0.0,0.000000,0.059996,...,0.224999,0.000000,0.124293,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,0.688887,0.580944,0.049997,0.022987,0.000000,0.477991,0.209302,0.0,0.025639,0.440004,...,0.563639,0.000000,0.197744,0.000000,0.100911,0.000000,0.000000,0.000000,0.000000,0.131038
986,0.022221,0.019046,0.137498,0.632180,0.017542,0.000000,0.000000,0.0,0.384616,0.039998,...,0.397729,0.000000,0.107347,0.000000,0.110089,0.000000,0.000000,0.002599,0.000000,0.151723
987,0.022221,0.019046,0.000000,0.126436,0.701743,0.012578,0.093018,0.0,0.051279,0.000000,...,0.011364,0.004833,0.028250,0.136583,0.155961,0.013513,0.000000,0.072727,0.000000,0.013792
988,0.000000,0.000000,0.300000,0.333339,0.087721,0.000000,0.000000,0.0,0.487174,0.019999,...,0.193181,0.149755,0.005652,0.014635,0.137605,0.000000,0.277413,0.000000,0.022470,0.020692


## Partie 2 : Modélisation et évaluation des données